# A State of Trance - Average Episode BPM
*How has A State of Trance changed over time?*

I'm not a coder, a data scientist, or a DJ. But with nothing better to do, I set out to answer this question using [JupyterLab](https://jupyterlab.readthedocs.io/en/stable/) uses the [Spotify Web API](https://developer.spotify.com/documentation/web-api/) ...

## Methodology

A quick call to the Spotify Web API can return over a dozen [audio features for a track](https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/), notably `tempo` - "The overall estimated tempo of a track in beats per minute (BPM)."

Conveniently, the entire back catalogue of A State of Trance - 950+ episodes - has been uploaded to Spotify under the artist ["Armin van Buuren ASOT Radio"](https://open.spotify.com/artist/25mFVpuABa9GkGcj9eOPce).

In [140]:
import os
import yaml
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

with open('spotipy_credentials.yaml', 'r') as spotipy_credentials_file:
    credentials = yaml.safe_load(spotipy_credentials_file)
    os.environ["SPOTIPY_CLIENT_ID"] = credentials['spotipy_credentials']['spotipy_client_id']
    os.environ["SPOTIPY_CLIENT_SECRET"] = credentials['spotipy_credentials']['spotipi_client_seret']

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

ASOT_SPOTIFY_ID = '25mFVpuABa9GkGcj9eOPce' # Armin van Buuren ASOT Radio (https://open.spotify.com/artist/25mFVpuABa9GkGcj9eOPce)

episodes = []
results = sp.artist_albums(ASOT_SPOTIFY_ID, album_type='album')
episodes.extend(results['items'])
while results['next']:
    results = sp.next(results)
    episodes.extend(results['items'])
seen = set()  # to avoid dups
for episode in episodes:
    name = episode['name']
    if name not in seen:
        seen.add(name)

episodes = episodes[::-1]

In [155]:
len(episodes)

933

In [154]:
import json
print(json.dumps(episodes[1], indent=2))

{
  "album_group": "album",
  "album_type": "album",
  "artists": [
    {
      "external_urls": {
        "spotify": "https://open.spotify.com/artist/25mFVpuABa9GkGcj9eOPce"
      },
      "href": "https://api.spotify.com/v1/artists/25mFVpuABa9GkGcj9eOPce",
      "id": "25mFVpuABa9GkGcj9eOPce",
      "name": "Armin van Buuren ASOT Radio",
      "type": "artist",
      "uri": "spotify:artist:25mFVpuABa9GkGcj9eOPce"
    }
  ],
  "available_markets": [
    "AD",
    "AE",
    "AR",
    "AT",
    "AU",
    "BE",
    "BG",
    "BH",
    "BO",
    "BR",
    "CA",
    "CH",
    "CL",
    "CO",
    "CR",
    "CY",
    "CZ",
    "DE",
    "DK",
    "DO",
    "DZ",
    "EC",
    "EE",
    "EG",
    "ES",
    "FI",
    "FR",
    "GB",
    "GR",
    "GT",
    "HK",
    "HN",
    "HU",
    "ID",
    "IE",
    "IL",
    "IN",
    "IS",
    "IT",
    "JO",
    "JP",
    "KW",
    "LB",
    "LI",
    "LT",
    "LU",
    "LV",
    "MA",
    "MC",
    "MT",
    "MX",
    "MY",
    "NI",
    "NL",
    "

In [157]:
episode_tracks = sp.album_tracks(episodes[1]['uri'])
print(json.dumps(episode_tracks, indent=2))

{
  "href": "https://api.spotify.com/v1/albums/2RLKYKweji0JBAi2OtxOCt/tracks?offset=0&limit=50",
  "items": [
    {
      "artists": [
        {
          "external_urls": {
            "spotify": "https://open.spotify.com/artist/0SfsnGyD8FpIN4U4WCkBZ5"
          },
          "href": "https://api.spotify.com/v1/artists/0SfsnGyD8FpIN4U4WCkBZ5",
          "id": "0SfsnGyD8FpIN4U4WCkBZ5",
          "name": "Armin van Buuren",
          "type": "artist",
          "uri": "spotify:artist:0SfsnGyD8FpIN4U4WCkBZ5"
        }
      ],
      "available_markets": [
        "AD",
        "AE",
        "AR",
        "AT",
        "AU",
        "BE",
        "BG",
        "BH",
        "BO",
        "BR",
        "CA",
        "CH",
        "CL",
        "CO",
        "CR",
        "CY",
        "CZ",
        "DE",
        "DK",
        "DO",
        "DZ",
        "EC",
        "EE",
        "EG",
        "ES",
        "FI",
        "FR",
        "GB",
        "GR",
        "GT",
        "HK",
       

In [165]:
audio_features = sp.audio_features(episode_tracks['items'][2]['uri'])
print(json.dumps(audio_features, indent=2))

[
  {
    "danceability": 0.588,
    "energy": 0.965,
    "key": 11,
    "loudness": -11.546,
    "mode": 0,
    "speechiness": 0.0587,
    "acousticness": 0.000847,
    "instrumentalness": 0.936,
    "liveness": 0.39,
    "valence": 0.45,
    "tempo": 133.053,
    "type": "audio_features",
    "id": "6OqkwRDlfdkMcs7P3TDn5Z",
    "uri": "spotify:track:6OqkwRDlfdkMcs7P3TDn5Z",
    "track_href": "https://api.spotify.com/v1/tracks/6OqkwRDlfdkMcs7P3TDn5Z",
    "analysis_url": "https://api.spotify.com/v1/audio-analysis/6OqkwRDlfdkMcs7P3TDn5Z",
    "duration_ms": 437156,
    "time_signature": 4
  }
]


In [95]:
import altair as alt
import numpy as np
import pandas as pd

In [167]:
bpm = []
for track in episode_tracks['items']:
    bpm.append(sp.audio_features(track['uri'])[0]['tempo'])

x = np.arange(len(episode_tracks['items']))   

source = pd.DataFrame({
  'track': x,
  'bpm': np.array(bpm)
})

alt.Chart(source).mark_line().encode(
    x='track',
    y='bpm'
)

alt.Chart(...)

In [246]:
episode_avg_bpm = []

for episode in episodes:
    try:
        episode_bpm = 0
        tracks_counted = 0
        for track in sp.album_tracks(episode['uri'])['items']:
            if "A State Of Trance" in track['name']:
                continue
            else:
                episode_bpm += sp.audio_features(track['uri'])[0]['tempo']
                tracks_counted += 1
        avg = episode_bpm/tracks_counted
        print(episode['name'], avg)
        episode_avg_bpm.append(avg)
    except:
        pass

A State Of Trance Episode 001 138.86957142857142
A State Of Trance Episode 003 135.295
A State Of Trance Episode 004 137.3078
A State Of Trance Episode 005 137.122875
A State Of Trance Episode 007 137.459125
A State Of Trance Episode 008 136.409
A State Of Trance Episode 009 138.58033333333333
A State Of Trance Episode 010 137.5172
A State Of Trance Episode 012 138.5285
A State Of Trance Episode 015 138.09266666666667
A State Of Trance Episode 018 136.7673333333333
A State Of Trance Episode 019 135.445
A State Of Trance Episode 025 138.4986
A State Of Trance Episode 028 (Top 20 Of 2001) 138.9054
A State Of Trance Episode 030 137.06583333333336
A State Of Trance Episode 031 139.99871428571427
A State Of Trance Episode 032 135.74475
A State Of Trance Episode 033 136.26457142857143
A State Of Trance Episode 035 138.22199999999998
A State Of Trance Episode 036 138.38516666666666
A State Of Trance Episode 037 137.13842857142856
A State Of Trance Episode 040 137.9286666666667
A State Of Tran

In [265]:
x = np.arange(len(episode_avg_bpm))   

source = pd.DataFrame({
  'episode': x,
  'avg bpm': np.array(episode_avg_bpm)
})

alt.Chart(source).mark_line().encode(
    alt.X('episode'),
    alt.Y('avg bpm', scale=alt.Scale(domain=(120, 150))),
).properties(
    title="A State of Trance - Average BPM of episode"
)

alt.Chart(...)

In [253]:
episode_bpm = 0
tracks_counted = 0
for track in sp.album_tracks('56qM5Y21wbvCW9l5GiAiaV')['items']:
    if "A State Of Trance" in track['name']:
        continue
    else:
        print(track['name'], ' - ', sp.audio_features(track['uri'])[0]['tempo'])
        episode_bpm += sp.audio_features(track['uri'])[0]['tempo']
        tracks_counted += 1
print(episode_bpm/tracks_counted)

Cloudwalking [ASOT 057] - Beatpusher Remix  -  141.61
Ligaya [ASOT 057] - Original Mix  -  141.784
Obsession [ASOT 057] - Original Mix  -  136.377
Gatex [ASOT 057] - Original Mix  -  184.848
Positron [ASOT 057] - Marco V Remix  -  139.264
148.7766


In [304]:
x = np.arange(len(episode_avg_bpm))

source = pd.DataFrame({
  'episode': x,
  'avg bpm': np.array(episode_avg_bpm)
})

source['138'] = 138

base = alt.Chart(source).mark_line().encode(
    alt.X('episode'),
    alt.Y('avg bpm', scale=alt.Scale(domain=(120, 150))),
).properties(
    title="A State of Trance - Average BPM of episode"
)

rule = alt.Chart(source).mark_rule(color='red').encode(
    y='138'
)

line + rule

alt.LayerChart(...)